In [ ]:
#| default_exp hit_endpoint_03

In [ ]:
#| export
import pandas as pd
import requests
import boto3
import base64
import json
from io import BytesIO
import joblib
import pickle
import numpy as np
pd.set_option('display.max_columns', None)

In [ ]:
#| export
def load_s3():
    df_pred = pd.read_csv('s3://spotify-net/for_prediction.csv', index_col=0)

    return df_pred

In [ ]:
#| export
def prep_frame(df):
    df_pred = df
    save_names = df_pred[['name', 'uri', 'artist']]
    df_pred = df_pred.drop(['name', 'uri', 'artist'], axis=1)
    df_pred = df_pred.sort_index(axis=1)
    pred_json = df_pred.to_json()

    return pred_json

In [ ]:
#| export
def make_request(endpoint):
    preds = requests.get(endpoint)
    preds = pd.read_json(preds.json()[0])

    return preds

Add to predicted playlists (how to prevent duplicates?)
Delete from net

In [ ]:
#| export
p_dict = {
    'DONNA':'0ZSKsOlj7QFX9LstS7fk39', 
    'Flowgurt':'1gM83cehv98l8cdMnZmk5N', 
    'Four K':'2b6mA2TxfNjdnKH7mvgZIh', 
    'Intermission Expedition':'4U9dVZ97mizvT2FBjx3fGx',
    'PANTHERA ONCA':'29U1jhpnjkwziC4pcsvpml', 
    'søndag':'2iDp3O7zAdEdeUzlZHTtFu'
}

In [ ]:
#| export
def cred():
    secret_name = "spotify_35"
    region_name = "us-east-2"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    get_secret_value_response = client.get_secret_value(SecretId=secret_name)
    client_id = json.loads(get_secret_value_response['SecretString'])['spot_clientID']
    client_secret = json.loads(get_secret_value_response['SecretString'])['spot_clientSECRET']
    access_token = json.loads(get_secret_value_response['SecretString'])['spot_ACC']
    refresh_token = json.loads(get_secret_value_response['SecretString'])['spot_REF']

    return client_id, client_secret, access_token, refresh_token

In [ ]:
#| export
def add_tracks(name_frame, client_id, client_secret, access_token, refresh_token):

    for track, prediction in zip(name_frame['uri'], name_frame['predictions']):
        target_playlist = p_dict[prediction]

        headers = {
                    'Authorization': f'Bearer {access_token}'
                            }

        ADD_URL = f'https://api.spotify.com/v1/playlists/{target_playlist}/tracks?uris={track}'
        r_add = requests.post(ADD_URL, headers=headers)

        if r_add.status_code == 401:

            TOKEN_URL = 'https://accounts.spotify.com/api/token'

            message = client_id + ':' + client_secret
            messageBytes = message.encode('ascii')
            base64Bytes = base64.b64encode(messageBytes)
            base64Message = base64Bytes.decode('ascii')
        
            headers = {
            'Authorization': 'Basic ' + base64Message,
            'Content-Type': 'application/x-www-form-urlencoded'
            }

            pars_refresh = {
                'grant_type': 'refresh_token',
                'refresh_token': refresh_token,
                'redirect_uri': 'http://localhost:8888/callback',
            }

            r_refresh = requests.post(TOKEN_URL, headers=headers, params=pars_refresh)
            access_token = r_refresh.json()['access_token']

            headers = {
            'Authorization': f'Bearer {access_token}'
                    }
            
            r_add = requests.post(ADD_URL, headers=headers)
    
    print('Added tracks to playlist')

In [ ]:
#| export
def delete_tracks(name_frame, client_id, client_secret, access_token, refresh_token):

    for track in name_frame['uri']:

        headers = {
                    'Authorization': f'Bearer {access_token}'
                            }

        delete_playlist = '3ubgXaHeBn1CWLUZPXvqkj'
        del_dict = {'tracks': [{'uri': track}]}

        DELETE_URL = f'https://api.spotify.com/v1/playlists/{delete_playlist}/tracks?uris={track}'
        r_delete = requests.delete(DELETE_URL, headers=headers, data=json.dumps(del_dict))

        if r_delete.status_code == 401:

            TOKEN_URL = 'https://accounts.spotify.com/api/token'

            message = client_id + ':' + client_secret
            messageBytes = message.encode('ascii')
            base64Bytes = base64.b64encode(messageBytes)
            base64Message = base64Bytes.decode('ascii')
        
            headers = {
            'Authorization': 'Basic ' + base64Message,
            'Content-Type': 'application/x-www-form-urlencoded'
            }

            pars_refresh = {
                'grant_type': 'refresh_token',
                'refresh_token': refresh_token,
                'redirect_uri': 'http://localhost:8888/callback',
            }

            r_refresh = requests.post(TOKEN_URL, headers=headers, params=pars_refresh)
            access_token = r_refresh.json()['access_token']

            headers = {
            'Authorization': f'Bearer {access_token}'
                    }
            
            r_delete = requests.delete(DELETE_URL, headers=headers, data=json.dumps(del_dict))
            print(r_delete.status_code)
    
    print('Removed tracks from net')

In [ ]:
#| export
if __name__ == '__main__':   
    df = load_s3()
    name_frame = df[['name', 'uri', 'artist']].copy()
    df_json = prep_frame(df)
    predictions = make_request(f'http://127.0.0.1:8000/model/{df_json}')
    name_frame['predictions'] = predictions
    name_frame = name_frame.iloc[:2]
    client_id, client_secret, access_token, refresh_token = cred()
    _ = add_tracks(name_frame, client_id, client_secret, access_token, refresh_token)
    _ = delete_tracks(name_frame, client_id, client_secret, access_token, refresh_token)

Added tracks to playlist
Removed tracks from net
